In [1]:
import pandas as pd
from datasketch import MinHash, MinHashLSH
from tqdm import tqdm
tqdm.pandas()

In [2]:
df1 = pd.read_csv('dataset_science_pt1.csv', encoding='utf-8')

In [4]:
len(df1)

2190

In [5]:
df2 = pd.read_csv('dataset_science_pt2.csv', encoding='utf-8')

In [6]:
len(df2)

1060

In [8]:
def hashes_simple(x):
    m1 = MinHash(num_perm=256)
    _ = [m1.update(token.encode('utf8')) for token in x.split()]
    return m1

In [9]:
%%time
texts = df2.text.to_list()
hashes = [hashes_simple(x) for x in texts]

Wall time: 1min 32s


In [10]:
lsh = MinHashLSH(threshold=0.8, num_perm=256)

i = 0

with lsh.insertion_session() as session:
    hashes_lsh = [('m'+str(u), x) for x, u in zip(hashes, range(i, i+len(hashes)+1))]
    for key, minhash in hashes_lsh:
            session.insert(key, minhash)

In [17]:
def filtr(x):
    h = hashes_simple(x)
    if len(lsh.query(h)):
        return False
    else:
        return True

In [20]:
df1['filt'] = df1.text.progress_apply(filtr)

100%|██████████████████████████████████████████████████████████████████████████████| 2190/2190 [03:55<00:00,  9.31it/s]


In [21]:
len(df1[df1['filt']])

1740

In [22]:
df1 = df1[df1['filt']].drop('filt', axis=1)
df1.to_csv('cleaned_dataset_science_pt1.csv',index=False, encoding='utf-8')